In [1]:
import os, dotenv

dotenv.load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["COMPOSIO_API_KEY"] = os.getenv("COMPOSIO_API_KEY")

## Calendar Agent Graph
<img src="../static/Google_Calendar_Agent.png" alt="Graph" width="600">

In [2]:
from langgraph.graph import MessagesState

class GoogleCalendarGraphState(MessagesState):
    pass


In [4]:
from langgraph.types import Command, interrupt
from langchain.chat_models import init_chat_model

gemini_model = init_chat_model("gemini-2.0-flash-exp", model_provider="google_genai")

In [6]:
from composio_langgraph import Action, ComposioToolSet

composio_toolset = ComposioToolSet()

calendar_write_tools = composio_toolset.get_tools(
    actions=[
        Action.GOOGLECALENDAR_CREATE_EVENT,
        Action.GOOGLECALENDAR_UPDATE_EVENT,
        Action.GOOGLECALENDAR_DELETE_EVENT,
        Action.GOOGLECALENDAR_QUICK_ADD,
    ]
)

calendar_read_tools = composio_toolset.get_tools(
    actions=[
        Action.GOOGLECALENDAR_FIND_EVENT,
        Action.GOOGLECALENDAR_FIND_FREE_SLOTS,
        Action.GOOGLECALENDAR_GET_CURRENT_DATE_TIME,
    ]
)

write_tools_by_name = {tool.name: tool for tool in calendar_write_tools}

In [7]:
write_tools_by_name

{'GOOGLECALENDAR_UPDATE_EVENT': StructuredTool(name='GOOGLECALENDAR_UPDATE_EVENT', description='Update An Existing Event In A Google Calendar.', args_schema=<class 'composio.utils.shared.UpdateEventRequest'>, func=<function ComposioToolSet._wrap_action.<locals>.function at 0x0000018B653B34C0>),
 'GOOGLECALENDAR_CREATE_EVENT': StructuredTool(name='GOOGLECALENDAR_CREATE_EVENT', description='Create A New Event In A Google Calendar.', args_schema=<class 'composio.utils.shared.CreateEventRequest'>, func=<function ComposioToolSet._wrap_action.<locals>.function at 0x0000018B653B2520>),
 'GOOGLECALENDAR_QUICK_ADD': StructuredTool(name='GOOGLECALENDAR_QUICK_ADD', description="Create A New Event In A Google Calendar Based On A Simple Text String Like 'Appointment At Somewhere On June 3Rd 10Am 10:25Am' You Can Only Give Title And Timeslot Here. No Recurring Meetings And No Attendee Can Be Added Here. This Is Not A Preferred Endpoint. Only Use This If No Other Endpoint Is Possible.", args_schema=<

In [8]:
agent_with_tools = gemini_model.bind_tools(calendar_write_tools + calendar_read_tools)

In [16]:
CALENDAR_AGENT_PROMPT = """
You are a highly capable AI assistant specializing in managing the user's Google Calendar with precision, clarity, and efficiency.

---
### Core Capabilities
You can assist with the following tasks:

1. **Finding Information:**
   - Locate specific events based on date, time, keywords, or attendees.
   - Identify the user's availability and free time slots within specified date ranges and durations.
   - Provide the current date and time contextually.

2. **Modifying the Calendar:**
   - Create new events with details including date, time, title, location, and description.
   - Delete existing events based on user confirmation.
   - Update events by modifying details such as date, time, title, location, or description.
   - Quickly add events based on natural language descriptions.

### Execution Guidelines
1. **Prioritize Clarity and Confirmation:** If the user's request is unclear or ambiguous, ask for clarification before proceeding.
2. **Interpret Date/Time Context Accurately:** Correctly resolve relative terms like "today," "tomorrow," "next Monday," or "end of the month" based on the user's time zone.
3. **Provide Comprehensive Event Details:** When retrieving events, include full details such as start time, end time, title, location, and description, if available.
4. **Execute Tasks in Logical Order:** If a task requires both reading and writing operations, ensure you fetch necessary data before modifying the calendar.
5. **Optimize User Interaction:** If a conflict is detected (e.g., scheduling an event over an existing commitment), notify the user and suggest alternative time slots.

### NOTE
If any function call/tool call fails, retry it 3 times. Make sure to handle tools efficiently.

---
**Important Details:**
- User Name: {user_name}
- User Email: {user_email}
- Today's date/time: {today_datetime}
- Timezone Offset (from UTC): {timezone_offset}

"""